# Assignment 4: Detecting and Mitigating Bias 

The goal of this tutorial is to introduce the basic functionality of AI Fairness 360 for detecting and mitigating bias. As before, we will work with the German Credit dataset. There are many metrics one can use to detect the presence of bias. Likewise, there are many different bias mitigation algorithms one can employ. AI Fairness 360 provides some of them most common metrics and algorithms.


### Bias mitigation techniques

We learnt about the different bias mitigation techniques in class called _pre-processing_, _in-processing_, and _post-processing_.


We will use AI Fairness 360 (`aif360`) to detect and mitigate bias. We will look for bias in the creation of a machine learning model that predicts whether an applicant should be given credit based on various features from a typical credit application. The protected attribute will be "Age", with "1" (older than or equal to 25) and "0" (younger than 25) being the values for the _privileged_ and _unprivileged_ groups, respectively.

In this notebook, we will:

1. Install and import packages and modules
2. Load dataset, split between train and test, and compute fairness metrics on original training dataset
3. Mitigate bias using a pre-processing algorithm (reweighing)
4. Mitigate bias using an in-processing algorithm (adversarial debiasing)
5. Mitigate bias using a post-processing algorithm (equalized odds post processing)


## 1. Import Statements

First, we install the necessary packages. Then we import several components from the `aif360` package. We are relying on aif360 for this assignment, so please start early to make sure that the dependencies are resolved and that the pacakges load correctly. 

In [ ]:
!pip install numba

In [ ]:
!pip install tensorflow-macos

In [ ]:
# No need to re-install if you already did so in Assignment 2
!pip install aif360

In [ ]:
# import all necessary packages
import numpy as np
np.random.seed(0)

from numba import jit

from aif360.datasets import GermanDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, DatasetMetric

from aif360.algorithms.preprocessing import Reweighing, LFR, DisparateImpactRemover
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing

from aif360.explainers import MetricTextExplainer, MetricJSONExplainer

from sklearn.linear_model import LogisticRegression

import tensorflow as tf
print(tf.__version__)

from IPython.display import Markdown, display

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import json
from collections import OrderedDict

## 2. Load Data, Specify Protected Attribute, and Split Data

We will use the German Credit data, set the protected attribute to be age, create two variables to represent the privileged and unprivileged groups, and split the original dataset into training and test data subsets. Finally, we will build a typical machine learning workflow that involves training a machine learning model on the training dataset and use a test dataset to assess the model's efficacy (e.g., accuracy, fairness). For this dataset, we have a binary classification problem that predicts individuals as being a good or a bad credit risk.

In this dataset, we consider older applicants (age >= 25) as the privileged group and younger applicants (age < 25) as the unprivileged group. 

We will use the preprocessed GermanDataset with one-hot encoded data provided by the aif360 package. 

In [ ]:
# note that we drop sex, which may also be a protected attribute
dataset_orig = GermanDataset(protected_attribute_names=['age'],
                             privileged_classes=[lambda x: x >= 25],
                             features_to_drop=['personal_status', 'sex'])

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [ ]:
print("Original data shape: ",dataset_orig.features.shape)
print("Train dataset shape: ", dataset_orig_train.features.shape)
print("Test dataset shape: ", dataset_orig_test.features.shape)

The object ```dataset_orig``` is an aif360 dataset, which has some useful methods and attributes that you can explore. More documentation is available at https://aif360.readthedocs.io/en/latest/modules/datasets.html. 
For now, we'll just transform the data into a pandas dataframe:

In [ ]:
df, dict_df = dataset_orig.convert_to_dataframe()
print("Shape: ", df.shape)
# print(df.columns)
# df.head(5)

## 3. Compute Fairness Metrics on Original Training Data
Now that we have identified the protected attribute "age" and defined privileged and unprivileged values, we can use aif360 to detect bias in the dataset.  

### Mean Outcomes

Compare the base rates (i.e., percentage of favorable results) for the privileged and unprivileged groups and report the difference (unprivileged base rate - privileged base rate). This is implemented in the ```mean_difference``` method on the BinaryLabelDatasetMetric class, as shown below:

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(
     dataset_orig_train, 
     unprivileged_groups=unprivileged_groups,
     privileged_groups=privileged_groups
  )
print("Original training dataset")
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

### Disparate Impact
We can calculate the ratio of (predicted) favorable outcomes for the unprivileged group compared to the privileged group as implemented in the ```disparate_impact``` method on the BinaryLabelDatasetMetric class:

In [ ]:
print("Original training dataset")
print("Disparate Impact = %f" % metric_orig_train.disparate_impact())

**Note:** The fairness metrics above will vary depending upon the train-test split. If the magnitude of mean difference is less than 10%, try another split.

### Built-In Explainers

```aif360``` has some useful explainers for the fairness metrics which can be used to interpret the fairness metric values:

In [ ]:
json_expl = MetricJSONExplainer(metric_orig_train)
def format_json(json_str):
    return json.dumps(json.loads(json_str, object_pairs_hook=OrderedDict),
                      indent=2)

Let's print the mean difference explainer:

In [ ]:
print(format_json(json_expl.mean_difference()))

We can also print the disparate impact explainer:

In [ ]:
print(format_json(json_expl.disparate_impact()))

**Q1:** Using the explainers above, tnterpret the difference in means and disparate impact in the German Credit data:

**Write your interpretation here**

### Build a model on the training data

Let's build a logistic regression model on this training data, predict credit risk for test data and compute the same fairness metrics over the model predictions.

In [ ]:
model = LogisticRegression(solver='liblinear', class_weight='balanced')

df_test, dict_df_test = dataset_orig_test.convert_to_dataframe()
df_train, dict_df_train = dataset_orig_train.convert_to_dataframe()

# Fit the model to the training data
x_train = df_train.drop(['credit'], axis=1)
y_train = df_train['credit']
model.fit(x_train, y_train)

x_test = df_test.drop(['credit'], axis=1)
y_test = df_test['credit']

y_pred = model.predict(x_test)

dataset_pred_test = dataset_orig_test.copy()
dataset_pred_test.labels = y_pred.copy()

metric_dataset_test = BinaryLabelDatasetMetric(
    dataset_pred_test, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
  )


In [ ]:
# write code here to compute fairness metrics

**Q2:** Using the fairness metric functions as before, report the bias observed in the model's predictions over test data. What do these values indicate? Are the model's predictions more biased or less biased compared to the bias observed in the training data?

**Write your answer here**

## 4. Bias Mitigation Techniques

We learnt in class that there are several bias mitigation techniques namely, pre-processing, in-processing, and post-processing algorithms.

_Pre-processing_ bias mitigation is performed at the data end, before the creation of the model. In other words, we transform the data such that a model learned on the transformed data produces less biased decisions.

_In-processing_ bias mitigation methods focus on the model training stage, as compared to pre-processing which focuses on transforming the data prior to model training. This suite of methods includes incorporating a fairness constraint during model training, tweaking the model's objective function, and adversarial learning.

_Post-processing_ bias mitigation focus on the model predictions after the model has been trained.



### 4.1 Bias Mitigation via Pre-Processing

AI Fairness 360 implements several pre-processing mitigation algorithms. We will use the **reweighing algorithm**, which is implemented in the `Reweighing` class in the `aif360.algorithms.preprocessing` package. As discussed in class, this algorithm will transform the dataset by assigning weights to instances in each (group, label) combination to change the base rates and ensure fairness before classification. The idea is to apply appropriate weights to different tuples in the training data to reduce discrimination with respect to the protected attributes.

You can find documentation for reweighting here:
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.Reweighing.html 

Call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (```dataset_transf_train```):

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

We can print the weights. Each observation in the data should have a weight. For brevity, let's look at the weights for the first 10 rows:

In [ ]:
len(dataset_transf_train.instance_weights)
dataset_transf_train.instance_weights[0:10]

### Compute Fairness Metrics in Transformed Data

We can check how effective the transformed data was in removing bias by calculating the metrics used for the original training dataset.

In [ ]:
metric_rw_train = BinaryLabelDatasetMetric(
    dataset_transf_train, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
  )

Print the difference in mean outcomes and disparate impact in the transformed data:

In [ ]:
# write your code here

**Q3:** How do these values compare to the difference in mean outcomes and disparate impact in the original data?

**Write your answer in this text cell:**

### Compute Fairness Metrics on Model Trained on Transformed Data

In the following, we will train a model on the transformed data and compute the metrics over predictions made on the test data.

**Q4:**  How do you expect the fairness metrics would be over a model trained on the transformed data?

**Write your answer in this text cell:**

Since the instances now have weights, we will use a classifier that can incorporate instance weights. In this case, we will use a Naive Bayes classifier (more details here: https://scikit-learn.org/stable/modules/naive_bayes.html). 

In [ ]:
df_train_rw, dict_df_train_rw = dataset_transf_train.convert_to_dataframe()

# Fit the model to the transformed training data
x_train_rw = df_train_rw.drop(['credit'], axis=1)
y_train_rw = df_train_rw['credit']

from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train_rw, y_train_rw)

# Use the model to make predictions on the test data
y_pred_rw = model.predict(x_test)

dataset_pred_test_rw = dataset_orig_test.copy()
dataset_pred_test_rw.labels = y_pred_rw.copy()

# Construct the BinaryLabelDatasetMetric object over the test predictions
metric_dataset_test_rw = BinaryLabelDatasetMetric(
    dataset_pred_test_rw, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
  )

# Print fairness metrics computed over test predictions
# write code here


**Q5:** Are your observations in line with what you expected in Q4 above? Why or why not?

**Write your answer in this text cell:**

**Q6:** Instead of reweighing, one could also apply techniques such as suppression, i.e. removing sensitive attributes. Write code below to train a model that does not use any information on the sensitive attribute, use this model to make predictions over the test data, and then compute the fairness metrics over the predictions.


In [ ]:
# write code here to implement suppression

**Q7:** Interpret your results. How does the preprocessing technique in Q5 compare to the suppression technique? 

**Write your answer in this text cell:**

### 4.2. Bias Mitigation via In-Processing

In-processing methods focus on the model training stage, as compared to pre-processing which focuses on transforming the data prior to model training. Broadly speaking, contemporary in-processing methods are stronger than pre-processing methods.

### Adversarial Debiasing

In this part of the notebook, we will use an in-processing algorithm, called _Adversarial Debiasing_, that we briefly discussed in class. From the aif360 documentation (https://aif360.readthedocs.io/en/v0.2.3/modules/inprocessing.html):

> Adversarial debiasing is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary’s ability to determine the protected attribute from the predictions. This approach leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit.

For intuition, you can think of adversarial debiasing as a model with two supervised learning tasks. The first task is to predict an outcome using the training data input. The second task, i.e. the adversary, is to predict a protected feature using these predictions and non-protected features in the training data input. The aim is to maximize the model's ability to carry out the first task (i.e. predict outcomes) while minimizing its ability to carry out the second task (i.e. predict protected features).

We implement adversarial debiasing below:

In [ ]:
# reset tensorflow graph
tf.compat.v1.reset_default_graph()

# start tensorflow session
sess = tf.compat.v1.Session()
tf.compat.v1.disable_eager_execution()

# create AdversarialDebiasing model
debiased_model = AdversarialDebiasing(
    privileged_groups = privileged_groups,
    unprivileged_groups = unprivileged_groups,
    scope_name = 'debiased_classifier',
    debias = True,
    sess = sess)

# fit the model to training data
debiased_model.fit(dataset_orig_train)

# make predictions on training and test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

# metrics
metric_dataset_debiasing_test = BinaryLabelDatasetMetric(
    dataset_debiasing_test, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
  )

# Close session
sess.close()

### Fairness Metrics under Adversarial Debiasing

The adversarial debiasing algorithm has built-in methods for the difference in mean outcomes (called ```.mean_difference()```) and disparate impact (called ```.disparate_impact()```). Print these below: 

In [ ]:
# write your code here

**Q8:** Interpret the difference in means and disparate impact for the predicted outcomes under adversarial debiasing. How do these compare to the metrics calculated in Q2 and Q5?

**Write your interpretation and comparison in this text cell:**

### 4.3. Bias Mitigation via Post-Processing

In this last section, we will use one of the post-processing algorithms in AI Fairness 360 called as **equalized odds postprocessing**, which is implemented in the `EqOddsPostprocessing` class in the `aif360.algorithms.postprocessing` package. This technique solves a linear program to find probabilities with which to change output labels to optimize equalized odds.

You can find documentation for reweighting here:
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.postprocessing.EqOddsPostprocessing.html 

Call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (```dataset_post_train```):

In [ ]:
df_test, dict_df_test = dataset_orig_test.convert_to_dataframe()
df_train, dict_df_train = dataset_orig_train.convert_to_dataframe()

# Fit the model to the training data and predict for test data
# write code here
# dataset_pred_test -- dataset with predictions stored in labels

# create Equalized Odds Post processing object
eo_post = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

# fit the object to training data
eo_post.fit(dataset_orig_test, dataset_pred_test)

# make predictions on test data
# write code here

# construct metrics object
# write code here

# compute fairnesss metrics 
# write code here

**Q9:** Interpret the difference in fairness metrics for the predicted outcomes under this post-processing technique. How do these compare to the metrics calculated in Q2, Q5 and Q8?

**Write your interpretation and comparison in this text cell:**

# Submitting this Assignment Notebook

Once complete, please submit your assignment notebook as an attachment under \"Assignments > Assignment 4\" on Brightspace. You can download a copy of your notebook using ```File > Download .ipynb```. Please ensure you submit the `.ipynb` file (and not a `.py` file)."